In [3]:
# when you have a complex project structure or multiple environments
# pip install python-dotenv
# from dotenv import find_dotenv, load_dotenv
# load_dotenv(find_dotenv())

In [70]:
import transformers
import sentence_transformers

from datasets import load_dataset

data = load_dataset("microsoft/ms_marco", "v1.1")
data_test = data['test']
ms_marco = data_test.select_columns(['passages', 'query', 'query_id'])
ms_marco

Dataset({
    features: ['passages', 'query', 'query_id'],
    num_rows: 9650
})

In [129]:
test_data = []
for i in range(20):
    test_data.append(
        {
            'query' : ms_marco['query'][i],        
            'passage_text': ms_marco['passages'][i]['passage_text']
        }
    )

for i in test_data:
    print(i['query'])

does human hair stop squirrels
what are the benefits of fossil fuels
what is a apothem
average cost for custom canopy
what is a hardware in a computer
edi logistics definition
why should recreational marijuana be illegal
what class are spiders in
where is the amur leopard found
how much does it cost to spayed my female rabbit
what does proteus mirabilis look like
is bougainvillea a shrub
is oxyfluorfen safe on cotton
what types of bacteria are found in the mouth
is self insurance insurance or a deductible
average wage of an actor
how often should men get a PSA test
what is financial assets
Starch and _____ are common polysaccharide carbohydrates found in plants.
is cypress community storage flooded


In [130]:
# Importing OpenAI API key
import os
import openai
import sys
from openai import OpenAI

OPENAI_KEY = os.environ['OPENAI_API_KEY']
client = OpenAI(api_key=OPENAI_KEY)

First, we created the most elaborate step-by-step translation prompt for the model.

In [116]:
SYSTEM_PROMPT_A = """
***TRANSLATION FROM ENGLISH TO SERBIAN***

**GOALS**

You are an expert translation system who knows English and Serbian. 
Your primary goal is to produce a high-quality, natural-sounding translation from English to Serbian. 
The translation should be contextually accurate, fluent, and adhere to the grammatical rules and lexicon of the Serbian language. 
Avoid literal translations and ensure the output reads naturally in Serbian.

**TRANSLATION INSTRUCTIONS**

When translating sentences from English to Serbian, follow these steps:
    1. Understanding the Context:
        - Carefully read the sentence in English.
        - Understand the context and meaning, including any nuanced or idiomatic expressions.
        - Translate the sentence in a way that conveys the same meaning and tone in Serbian.
    2. Translation guidlines:
        - Terminology:
            - Translate terms accurately and consistently using appropriate Serbian terminology.
            - Avoid using English terminology where Serbian equivalents exist.
        - Name Translation:
            - Apply these instructions ONLY to the names of people and geographical entities:
                - Transcribe personal names according to Serbian language rules.
                - Translate names of cities, states, and geographical entities to their Serbian equivalents.
                - Leave company names, movie titles, song titles, and band names in their original English form.
        - Numeric Data Integrity:
            - Retain all numeric data, including decimal points and values, as they appear in the original.
            - Ensure that units of measurement are translated accurately and consistently with Serbian conventions.
        - Context Preservation:
            - Preserve the original sentence's context to avoid misinterpretation.
            - Ensure that the translation maintains the original meaning and intent.
        - Consistency: 
            - Maintain consistency in terminology, units, and formatting throughout the text.
        - Grammatical Accuracy:
            - Follow the grammatical rules of Serbian, including the correct declension of nouns, adjectives, and pronouns.
            - Choose synonyms that best fit the context, ensuring the translation is grammatically correct and natural-sounding.
            - Translate terms and their acronyms into Serbian.
            - Avoid translating English constructions directly to Serbian. 
            - Use Serbian lexicon and syntax.
        3. Proofreading: 
            - Make sure to proofread the translted text in Serbian and correct any mistakes.

   **FORMATTING INSTRUCTIONS**

   1. Strings should be enclosed within single quotation marks (''). 
   2. Use double quotes for strings and escape internal quotes with a backslashes (\).
   
   **OUTPUT FORMATTING**

   - The output must be valid JSON file.
   - The output must be parsable by Python's 'json.loads()' function.
   - Ensure consistent JSON formatting as illustrated in the example below:

       **EXAMPLE**
       {'translations' : [{'query' : 'This is a query',
                       ''passage_text ' : {'This is one passage. With another sentence.', 
                                           'This is yet another passage. With yet another sentence.'}]
       }
        **END OF EXAMPLE**

    - Strictly follow the structure provided in the example when generating the output.    
    - Make sure to translate both query and passage text for each example.

"""

Next, we made the same prompt more concise under the supposition it will be easier for the model to understand and result in a better translation.

In [115]:
SYSTEM_PROMPT_B = """
***TRANSLATION FROM ENGLISH TO SERBIAN***

**GOALS**

You are a professional translator fluent in English and Serbian. 
Your primary goal is to produce a high-quality, natural-sounding translation from English to Serbian. 
You are translating texts and questions pertaining to the texts. The translation is intended for dataset creation. Look at the example below:

***TRANSLATION EXAMPLE***
***ENGLISH***
query: 'What is a unicorn?'
passage_text: 'The unicorn is a legendary creature that has been described since antiquity as a beast with a single large, pointed, spiraling horn projecting from its forehead.'

***SERBIAN TRANSLATION***
query: 'Šta je jednorog?'
passage_text: 'Jednorog je mitsko stvorenje koje se od davnina opisuje kao zver sa jednim velikim, šiljastim, spiralnim rogom koji mu viri iz čela.'
***END OF TRANSLATION EXAMPLE***

To translate, follow the steps below:
   **TRANSLATION INSTRUCTIONS**
   1. Read and understand the sentence in English.
   2. When you understand the English sentence, start to translate.
   3. Pay close attention to both left and right context when you are making translation decisions.  4. Convey the original context, tone and meaning in the Serbian translation.
   4. Avoid literal translations and ensure the output reads naturally in Serbian.
   5. The translation must be contextually accurate, fluent, and adhere to the grammatical rules and lexicon of the Serbian language.
   6. The declination of nouns, adjectives, and pronouns must be correct.
   7. Make sure to proofread the translated text in Serbian and revise any mistakes. If no revisions are needed, provide the translations as they are.

   **FORMATTING INSTRUCTIONS**
   1. Strings should be enclosed within single quotation marks ('').
   2. Use double quotes for strings and escape internal quotes with a backslashes (\).
 
   **OUTPUT FORMATTING**
   1. The output must be a valid JSON file.
   2. The output must be parsable by Python's 'json.loads()' function.
   3. Ensure consistent JSON formatting as illustrated in the example below:

      **EXAMPLE**
      {'translations' : [{'query' : 'This is a query',
                      ''passage_text ' : {'This is one passage. With another sentence.',
                                          'This is yet another passage. With yet another sentence.'}]
      }
       **END OF EXAMPLE**

   - Strictly follow the structure provided in the example when generating the output.   
   - Make sure to translate text under both "query" and "passage_text" keys.

"""


Lastly, we created a "baseline" prompt, which provides straightforward instructions to the model, to see whether our "engineered" prompts make any difference in the quality of the translation. Evaluation of the translation quality is done both manually and automatically, by calculating cosine similarity between the embeddings of model's translation using different prompts.

In [75]:
SYSTEM_PROMPT_C = "You are an expert tarnslation system. Translate text from English to Serbian. Generate output in a readable JSON format Name keys: translations and passage_text"

In [76]:
def translate(query: str, prompt : str, model: str = 'gpt-3.5-turbo-0125') -> str:
  """ Translate to Serbian via OPENAI API. """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": prompt
      },
      {
        "role": "user",
        "content": query
      }
    ],
    temperature=0,
    top_p=1,
    response_format = {'type': "json_object"}
  )

  return response

In [12]:
import json
	
test_example = {'query' : 'does human hair stop squirrels',
                'passage_text': ['We have been feeding our back yard squirrels for the fall and winter and we noticed that a few of them have missing fur. One has a patch missing down his back and under both arms. Also another has some missing on his whole chest. They are all eating and seem to have a good appetite.',
                                 'Critters cannot stand the smell of human hair, so sprinkling a barrier of hair clippings around your garden, or lightly working it into the soil when you plant bulbs, apparently does have some merit. The whole thing kind of makes me laugh. It never occurred to me that we are the ones that stink.',
                                 "Spread some human hair around your vegetable and flower gardens. This will scare the squirrels away because humans are predators of squirrels. It is better if the hair hasn't been washed so the squirrels will easily pick up the human scent.",
                                 '1 You can sprinkle blood meal around your garden as well. 2  Don’t trap and relocate squirrels. 3  This is a losing battle since the population of squirrels is extremely high. 4  Also, if the animal is a female there is a high likelihood that you will remove her from babies that depend on her for survival.',
                                 'Hair loss in squirrels can be caused by mange or fungal disease. Mange is a disease caused by microscopic mites that burrow under the skin. The squirrel mange mite has been reported in both fox and gray squirrels. There have been no reports of this type of mange being transmitted to humans or domestic pets.',
                                 '1 You can also scatter dog or human hair around your garden. 2  One readers shares, “I used to have a problem with squirrels digging up my bulbs. 3  Now, once in the spring and once in the fall, I ask my hairdresser to save a big bag of hair for me. 4  I lightly dig this into the soil.',
                                 'Rabbit repellent. Human hair will keep rabbits out of your garden!! Collect hair from your brushes and spread it around your garden! (via Pioneer Thinking). Natural mulch. When woven into a mat, it retains moisture, it deters weeds. In some instances, it can actually reduce soil erosion. (via NPR). Plant fertilizer.']
}

test_mini = 'Critters cannot stand the smell of human hair, so sprinkling a barrier of hair clippings around your garden, or lightly working it into the soil when you plant bulbs, apparently does have some merit. The whole thing kind of makes me laugh. It never occurred to me that we are the ones that stink.'

json_example = json.dumps(test_example)
json_example

'{"query": "does human hair stop squirrels", "passage_text": ["We have been feeding our back yard squirrels for the fall and winter and we noticed that a few of them have missing fur. One has a patch missing down his back and under both arms. Also another has some missing on his whole chest. They are all eating and seem to have a good appetite.", "Critters cannot stand the smell of human hair, so sprinkling a barrier of hair clippings around your garden, or lightly working it into the soil when you plant bulbs, apparently does have some merit. The whole thing kind of makes me laugh. It never occurred to me that we are the ones that stink.", "Spread some human hair around your vegetable and flower gardens. This will scare the squirrels away because humans are predators of squirrels. It is better if the hair hasn\'t been washed so the squirrels will easily pick up the human scent.", "1 You can sprinkle blood meal around your garden as well. 2  Don\\u2019t trap and relocate squirrels. 3  

In [77]:
def get_results(results):
    result = json.loads(results.choices[0].message.content)
    return result

In [47]:
translate_b_test = translate(json_example, SYSTEM_PROMPT_B, model='gpt-3.5-turbo-0125')
get_results(translate_b_test)

{'translations': [{'query': 'da li ljudska kosa zaustavlja veverice',
   'passage_text': ['Hranimo naše veverice u dvorištu tokom jeseni i zime i primetili smo da nekoliko njih ima ćelave delove. Jedna ima ćelavu fleku duž leđa i ispod oba krila. Takođe, druga ima nekoliko ćelavih delova na celom grudnom košu. Sve one jedu i čini se da imaju dobar apetit.',
    'Životinje ne podnose miris ljudske kose, pa posipanje barijere od isečene kose oko vaše bašte, ili lagano unošenje u zemlju prilikom sadnje lukovica, očigledno ima neku vrednost. Čitava stvar me nekako nasmejava. Nikada mi nije palo na pamet da smo mi ti koji smrde.',
    'Rasporedite malo ljudske kose oko vaših bašti sa povrćem i cvećem. To će uplašiti veverice jer su ljudi predatori veverica. Bolje je ako kosa nije oprana kako bi veverice lako uhvatile ljudski miris.',
    '1 Takođe možete posuti krvavu brašnu oko vaše bašte. 2 Ne hvatajte i ne premeštajte veverice. 3 Ovo je gubitna bitka jer je populacija veverica izuzetno v

In [14]:
translate_a = translate(json_example, SYSTEM_PROMPT_A, model='gpt-3.5-turbo-0125')
translate_b = translate(json_example, SYSTEM_PROMPT_B, model='gpt-3.5-turbo-0125')
translate_c = translate(json_example, SYSTEM_PROMPT_C, model='gpt-3.5-turbo-0125')

results_a = get_results(translate_a)
results_b = get_results(translate_b)
results_c = get_results(translate_c)

print(results_a)
print('____________________________')
print(results_b)
print('____________________________')
print(results_c)

{'translations': [{'query': 'da li ljudska kosa zaustavlja veverice', 'passage_text': ['Hranimo naše veverice u dvorištu tokom jeseni i zime i primetili smo da nekoliko njih ima nedostatak dlake. Jedna ima mesto bez dlake duž leđa i ispod oba krila. Takođe, druga ima nedostatak na celom grudnom košu. Sve one jedu i čini se da imaju dobar apetit.', 'Životinje ne podnose miris ljudske kose, pa posipanje barijere od isečene kose oko vaše bašte, ili lagano unošenje u zemlju prilikom sadnje lukovica, očigledno ima neku vrednost. Čitava stvar me nekako nasmejava. Nikada mi nije palo na pamet da smo mi ti koji smrde.', 'Rasporedite malo ljudske kose oko vaših bašti sa povrćem i cvećem. To će uplašiti veverice jer su ljudi predator veverica. Bolje je ako kosa nije oprana tako da će veverice lako uhvatiti ljudski miris.', '1 Možete posuti krvavu brašnu oko vaše bašte takođe. 2 Ne hvatajte i ne premeštajte veverice. 3 Ovo je gubitnička bitka jer je populacija veverica izuzetno visoka. 4 Takođe, 

In [15]:
def get_embeddings(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")

    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [16]:
embeddings_a = []

for i in results_a['translations'][0]['passage_text']:
    embedding = get_embeddings(i)
    embeddings_a.append(embedding)

print(embeddings_a[0])
print(len(embeddings_a))

[0.006139735691249371, -0.003383054630830884, -0.008635692298412323, -0.030218563973903656, -0.0025420603342354298, 0.0014888301957398653, -0.034288402646780014, -0.016902554780244827, -0.014689579606056213, -0.020120272412896156, 0.012502040714025497, 0.013494064100086689, -0.016075870022177696, 0.004540415480732918, -0.006378202699124813, 0.027140747755765915, 0.034873444586992264, -0.012266753241419792, 0.028005588799715042, -0.012940820306539536, 0.009837566874921322, 0.007077706512063742, -0.013163389638066292, -0.0020285609643906355, 0.0020285609643906355, -0.030218563973903656, 0.023910311982035637, 0.003936298657208681, -0.010886822827160358, -0.003127099946141243, 0.017754677683115005, -0.01122385635972023, -0.034034039825201035, -0.00584721565246582, -0.0225494597107172, -0.013125235214829445, 0.008088807575404644, 0.0004491132276598364, 0.009036317467689514, 0.002529342193156481, 0.002934736432507634, -0.0020683056209236383, -0.012673737481236458, 0.003529314650222659, -0.01

In [17]:
embeddings_b = []

for i in results_b['translations'][0]['passage_text']:
    embedding = get_embeddings(i)
    embeddings_b.append(embedding)

print(embeddings_b[0])
print(len(embeddings_b))

[0.0100962920114398, -0.0031709459144622087, -0.013977529481053352, -0.030796226114034653, 0.0038463573437184095, -0.0021483157761394978, -0.034575995057821274, -0.011643713340163231, -0.013711170293390751, -0.02468264289200306, 0.0069633973762393, 0.01185933779925108, -0.01577862724661827, -0.003459502011537552, -0.0014427803689613938, 0.029959097504615784, 0.03668150305747986, -0.004372734576463699, 0.018201230093836784, -0.02942637726664543, 0.011415405198931694, 0.003979537170380354, -0.017972921952605247, -0.000987749663181603, -0.005165470764040947, -0.024733377620577812, 0.017148476094007492, 0.004518597852438688, -0.0026081029791384935, -0.007787843234837055, 0.011935439892113209, -0.011910072527825832, -0.032369013875722885, -0.011967149563133717, -0.029680052772164345, -0.010793900117278099, 0.017326047644019127, -0.0021530722733587027, -0.0005771121359430254, 0.024987053126096725, 0.01156126894056797, -0.002481265226379037, -0.014256572350859642, 0.003713177749887109, -0.008

In [19]:
embeddings_c = []

for i in results_c['translations']['passage_text']:
    embedding = get_embeddings(i)
    embeddings_c.append(embedding)

print(embeddings_c[0])
print(len(embeddings_c))

[0.0012461671140044928, -0.0054005905985832214, -0.007429011631757021, -0.028717832639813423, -0.002062014536932111, -0.00011527845344971865, -0.03340175747871399, -0.017737485468387604, -0.019938673824071884, -0.019209209829568863, 0.012074543163180351, 0.017443139106035233, -0.017711888998746872, 0.009099099785089493, -0.0060628666542470455, 0.025352060794830322, 0.03606365993618965, -0.007345827296376228, 0.03355532884597778, -0.010551627725362778, 0.006865916773676872, 0.0028874604031443596, -0.010033324360847473, -0.007422612514346838, -0.001903644180856645, -0.02866664156317711, 0.022178255021572113, 0.007768148090690374, 0.00025955148157663643, -0.0032649897038936615, 0.01242647785693407, -0.011492252349853516, -0.035321399569511414, -0.011549841612577438, -0.02302289754152298, -0.0077489516697824, 0.007320231758058071, 0.0015397121896967292, 0.005806914530694485, 0.00025715192896313965, 0.007134666666388512, -0.0052886116318404675, -0.011914573609828949, 0.005682137794792652, -

In [20]:
import numpy as np
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

def get_similarity(embedding1, embedding2):
    v1 = np.array(embedding1)
    v2 = np.array(embedding2)
    similarity = cosine_similarity(v1.reshape(1, -1), v2.reshape(1, -1))[0][0]
    
    return similarity

In [21]:
similarity_a_b = get_similarity(embeddings_a, embeddings_b)
similarity_a_c = get_similarity(embeddings_a, embeddings_c)
similarity_b_c = get_similarity(embeddings_b, embeddings_c)

print(f"Similarity of Prompt A and Prompt B embeddings: {similarity_a_b}")
print(f"Similarity of Prompt A and Prompt C embeddings: {similarity_a_c}")
print(f"Similarity of Prompt B and Prompt C embeddings: {similarity_b_c}")

Similarity of Prompt A and Prompt B embeddings: 0.9870178017575935
Similarity of Prompt A and Prompt C embeddings: 0.9894435039266004
Similarity of Prompt B and Prompt C embeddings: 0.9871670963592226


In [87]:
test_example = json.dumps(test_data)
translate_test = translate(json_example, SYSTEM_PROMPT_B, model='gpt-3.5-turbo-0125')
results_test = get_results(translate_test)
results_test

{'translations': [{'query': '', 'passage_text': ''}]}

In [88]:
test_example

'[{"query": "does human hair stop squirrels", "passage_text": ["We have been feeding our back yard squirrels for the fall and winter and we noticed that a few of them have missing fur. One has a patch missing down his back and under both arms. Also another has some missing on his whole chest. They are all eating and seem to have a good appetite.", "Critters cannot stand the smell of human hair, so sprinkling a barrier of hair clippings around your garden, or lightly working it into the soil when you plant bulbs, apparently does have some merit. The whole thing kind of makes me laugh. It never occurred to me that we are the ones that stink.", "Spread some human hair around your vegetable and flower gardens. This will scare the squirrels away because humans are predators of squirrels. It is better if the hair hasn\'t been washed so the squirrels will easily pick up the human scent.", "1 You can sprinkle blood meal around your garden as well. 2  Don\\u2019t trap and relocate squirrels. 3 

In [131]:
def translate_examples(example, prompt):
    results = []
    for t in example:
        json_example = json.dumps(t)
        translate_example = translate(json_example, prompt, model='gpt-3.5-turbo-0125')
        results_example = json.loads(translate_example.choices[0].message.content)
        results.append(results_example)
    
    return results

In [133]:
len(test_data)

20

In [135]:
test_translate_a = translate_examples(test_data, SYSTEM_PROMPT_A)

In [136]:
test_translate_a

[{'translations': [{'query': 'da li ljudska kosa zaustavlja veverice',
    'passage_text': ['Hranili smo naše veverice u dvorištu tokom jeseni i zime i primetili smo da nekoliko njih ima ćelave delove. Jedna ima mesto bez dlake duž leđa i ispod oba ramena. Takođe, druga ima nedostatak dlake na celom grudnom košu. Sve one jedu i čini se da imaju dobar apetit.',
     'Životinje ne podnose miris ljudske kose, pa posipanje barijere od isečene kose oko vaše bašte, ili lagano unošenje u zemlju prilikom sadnje lukovica, očigledno ima neku vrednost. Čitava stvar me nekako nasmejava. Nikada mi nije palo na pamet da smo mi ti koji smrde.',
     'Rasporedite malo ljudske kose oko vaših povrtnjaka i cvećnjaka. To će uplašiti veverice jer su ljudi predator veverica. Bolje je ako kosa nije oprana tako da će veverice lako uhvatiti ljudski miris.',
     '1 Možete posuti krvavu brašnu oko vaše bašte takođe. 2 Ne hvatajte i ne premeštajte veverice. 3 Ovo je gubitna bitka jer je populacija veverica izuze

In [137]:
test_translate_b = translate_examples(test_data, SYSTEM_PROMPT_B)

In [138]:
test_translate_b

[{'translations': [{'query': 'da li ljudska kosa zaustavlja veverice',
    'passage_text': ['Hranimo naše veverice u dvorištu tokom jeseni i zime i primetili smo da nekoliko njih ima nedostatak dlake. Jedna ima mesto bez dlake duž leđa i ispod oba krila. Takođe, druga ima nedostatak dlake na celom grudnom košu. Sve one jedu i čini se da imaju dobar apetit.',
     'Životinje ne podnose miris ljudske kose, pa posipanje barijere od isečene kose oko vaše bašte, ili lagano unošenje u zemlju prilikom sadnje lukovica, očigledno ima neku vrednost. Čitava stvar me nekako natera da se nasmejem. Nikada mi nije palo na pamet da smo mi ti koji smrde.',
     'Rasporedite malo ljudske kose oko vaših povrtnjaka i cvećnjaka. To će uplašiti veverice jer su ljudi predator veverica. Bolje je ako kosa nije oprana tako da će veverice lako uhvatiti ljudski miris.',
     '1 Takođe možete posuti krvavu brašnu oko vaše bašte. 2 Ne hvatajte i ne premeštajte veverice. 3 Ovo je gubitna bitka jer je populacija veve

In [149]:
test_translate_c = translate_examples(test_data, SYSTEM_PROMPT_C)

In [198]:
test_translate_c

[{'translations': {'passage_text': ['Hranimo naše veverice u dvorištu tokom jeseni i zime i primetili smo da nekoliko njih ima izgubljenu dlaku. Jedna ima mesto bez dlake duž leđa i ispod oba ramena. Takođe, druga ima nedostatak dlake na celom grudnom košu. Sve one jedu i čini se da imaju dobar apetit.',
    'Životinje ne podnose miris ljudske dlake, pa posipanje barijere od isečene dlake oko vaše bašte, ili lagano raditi u zemlju kada sadite lukovice, očigledno ima neku vrednost. Čitava stvar me nekako natera da se nasmejem. Nikada mi nije palo na pamet da smo mi ti koji smrde.',
    'Rasporedite nešto ljudske dlake oko vaših povrtnih i cvećnih bašta. To će oterati veverice jer su ljudi predator veverica. Bolje je ako dlaka nije oprana tako da će veverice lako pokupiti ljudski miris.',
    '1 Možete posuti krvavu obrok oko vaše bašte takođe. 2 Ne uhvatite i ne premeštajte veverice. 3 Ovo je gubitna bitka jer je populacija veverica izuzetno visoka. 4 Takođe, ako je životinja ženka, pos

In [189]:
import pandas as pd
def save_results(data):
    rows = []
    for entry in data:
        translation = entry['translations'][0]  # Assuming there's always one translation per entry
        query = translation['query']
        for text in translation['passage_text']:
            rows.append({'query': query, 'passage_text': text})
    df = pd.DataFrame(rows)
    return df

In [199]:
df_a = save_results(test_translate_a)
df_b = save_results(test_translate_b)
# different keys, it saves translation : translated as key-value pairs
# does not translate most prompts, ommited from manual evaluation
#df_c = save_results(test_translate_c)

df_a.to_csv('~/Development/SRBedding/translation-pipeline-test/testing_data/test_a.csv', index=False)
df_b.to_csv('~/Development/SRBedding/translation-pipeline-test/testing_data/test_b.csv', index=False)
#df_c.to_csv('~/Development/SRBedding/translation-pipeline-test/testing_data/test_c.csv', index=False)

Decision after prompt evaluation: SYSTEM_PROMPT_B

## API parallel requests

In [ ]:
# mora skripta ne radi sa notebook-om